In [ ]:
import sys
print(sys.executable)

In [ ]:
import os
os.chdir('C:/Users/Shubham/Hand-Gesture-Recognition-for-HCI-main/')

In [ ]:
# !pip install hyperopt
# !pip install protobuf
# !pip install keras-preprocessing
!pip install keras

In [1]:
import os
from lib.kerashyper.kerashypetune import KerasGridSearch
from keras.optimizers import SGD

from lib.data_loader import DataLoader
from lib.image import ImageDataGenerator
from lib.resnet_model import Resnet3DBuilder

In [2]:
!tar chvfz notebook.tar.gz *

In [3]:
root_directory = r'C:\Users\Shubham\Hand-Gesture-Recognition-for-HCI-main\dataset'
labels_csv_path = os.path.join(root_directory, 'labels_extracted.csv')
train_csv_path = os.path.join(root_directory, 'train_extracted.csv')
val_csv_path = os.path.join(root_directory, 'validation_extracted.csv')
train_path = os.path.join(root_directory, 'Train')
val_path = os.path.join(root_directory, 'Validation')

data = DataLoader(labels_csv_path, train_csv_path, val_csv_path)

In [4]:
train_path

'C:\\Users\\Shubham\\Hand-Gesture-Recognition-for-HCI-main\\dataset\\Train'

In [5]:
WIDTH = 96
HEIGHT = 64
TARGET_SIZE = (HEIGHT, WIDTH)
BATCH_SIZE = 32
N_FRAMES = 16
SKIP = 1
INPUT_SHAPE = (N_FRAMES,) + TARGET_SIZE + (3,)
N_CLASSES = 10
EPOCHS = 1
WORKERS = 2

param_grid = {
    'drop_rate': [0.2, 0.5],
    'learning_rate': [0.001, 0.01]
}

In [6]:
def define_model(params):

    model = Resnet3DBuilder.build_resnet_101(
        input_shape=INPUT_SHAPE, 
        num_outputs=N_CLASSES,
        drop_rate=params['drop_rate']
    )

    optimizer = SGD(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [7]:
image_datagen = ImageDataGenerator()

train_generator = image_datagen.flow_video_from_dataframe(
    dataframe=data.train_df, 
    directory=train_path, 
    path_classes=labels_csv_path, 
    x_col='video_id', 
    y_col='label', 
    target_size=TARGET_SIZE, 
    class_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE, 
    shuffle=True,
    seed=42,
    nb_frames=N_FRAMES, 
    skip=SKIP
)

validation_generator = image_datagen.flow_video_from_dataframe(
    dataframe=data.val_df, 
    directory=val_path, 
    path_classes=labels_csv_path, 
    x_col='video_id', 
    y_col='label', 
    target_size=TARGET_SIZE, 
    class_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE, 
    shuffle=True,
    seed=42,
    nb_frames=N_FRAMES, 
    skip=SKIP
)

Found 8000 video folders belonging to 10 classes.
Found 2000 video folders belonging to 10 classes.


In [8]:
kgs = KerasGridSearch(
    hypermodel=define_model, 
    param_grid=param_grid,
    monitor='val_accuracy', 
    greater_is_better=True, 
    tuner_verbose=1
)

n_sample_train = len(data.train_df)
n_sample_val = len(data.val_df)

kgs.search(
    x=train_generator, 
    steps_per_epoch=n_sample_train/BATCH_SIZE,
    validation_data=validation_generator, 
    validation_steps=n_sample_val/BATCH_SIZE,
    epochs=1, 
    workers=WORKERS,
    verbose=1
)


4 trials detected for ('drop_rate', 'learning_rate')

***** (1/4) *****
Search({'drop_rate': 0.2, 'learning_rate': 0.01})
250/250 [==============================] - 6980s 28s/step - loss: 12.8840 - accuracy: 0.1124 - val_loss: 12.8311 - val_accuracy: 0.1220
SCORE: 0.122 at epoch 1

***** (2/4) *****
Search({'drop_rate': 0.2, 'learning_rate': 0.001})
250/250 [==============================] - 7354s 29s/step - loss: 12.8970 - accuracy: 0.1000 - val_loss: 12.8494 - val_accuracy: 0.1195
SCORE: 0.1195 at epoch 1

***** (3/4) *****
Search({'drop_rate': 0.5, 'learning_rate': 0.01})
250/250 [==============================] - 5918s 24s/step - loss: 12.9096 - accuracy: 0.1010 - val_loss: 12.8343 - val_accuracy: 0.1055
SCORE: 0.1055 at epoch 1

***** (4/4) *****
Search({'drop_rate': 0.5, 'learning_rate': 0.001})
250/250 [==============================] - 9134s 36s/step - loss: 12.9044 - accuracy: 0.0997 - val_loss: 12.8425 - val_accuracy: 0.0985
SCORE: 0.0985 at epoch 1


In [ ]:
# grid_search.fit(X_train, y_train)

In [9]:
kgs.best_params

{'drop_rate': 0.2,
 'learning_rate': 0.01,
 'epochs': 1,
 'steps_per_epoch': 250.0,
 'batch_size': None}